## 四、Vectorstores

向量数据库是用于高效计算和管理大量向量数据的解决方案。向量数据库是一种专门用于存储和检索向量数据（embedding）的数据库系统。它与传统的基于关系模型的数据库不同，它主要关注的是向量数据的特性和相似性。

在向量数据库中，数据被表示为向量形式，每个向量代表一个数据项。这些向量可以是数字、文本、图像或其他类型的数据。向量数据库使用高效的索引和查询算法来加速向量数据的存储和检索过程。

Langchain 集成了超过 30 个不同的向量存储库。我们选择 Chroma 是因为它轻量级且数据存储在内存中，这使得它非常容易启动和开始使用。

In [ ]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline

In [ ]:
# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
loader = PyMuPDFLoader("../docs/LeeDL-Tutorial/LeeDL_Tutorial.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pages = loader.load()
# 切分文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
docs = text_splitter.split_documents(documents)
# 定义 Embeddings
embedding = OpenAIEmbeddings()

embeddings = OpenAIEmbeddings() 
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)
llm = HuggingFacePipeline(pipeline=generate_text)
# checking again that everything is working fine
llm(prompt="Explain me the difference between Data Lakehouse and Data Warehouse.")


In [ ]:
persist_directory = 'docs/chroma/leedl/'


In [ ]:
!rm -rf './docs/chroma/leedl'  # 删除旧的数据库文件（如果文件夹中有文件的话），window电脑请手动删除

构建 Chroma 向量库

In [ ]:
vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embedding,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)

大家也可以直接载入已经构建好的向量库

In [ ]:
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

In [ ]:
sim_docs = vectordb.similarity_search(question,k=3)
print(f"检索到的内容数：{len(sim_docs)}")

In [ ]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容：{sim_doc.page_content[:100]}")

In [ ]:
如果只考虑检索出内容的相关性会导致内容过于单一，可能丢失重要信息。

最大边际相关性 (`MMR, Maximum marginal relevance`) 可以帮助我们在保持相关性的同时，增加内容的丰富度。


核心思想是在已经选择了一个相关性高的文档之后，再选择一个与已选文档相关性较低但是信息丰富的文档。这样可以在保持相关性的同时，增加内容的多样性，避免过于单一的结果。

In [1]:
mmr_docs = vectordb.max_marginal_relevance_search(question,k=3)

NameError: name 'vectordb' is not defined

In [ ]:
for i, sim_doc in enumerate(mmr_docs):
    print(f"通过 MMR 检索到的第{i}个内容：{sim_doc.page_content[:100]}")

可以看到多内容有了更多的差异。

在此之后，我们要确保通过运行 vectordb.persist 来持久化向量数据库，以便我们在未来的课程中使用。

让我们保存它，以便以后使用！

In [ ]:
vectordb.persist()

NameError: name 'vectordb' is not defined

## 四、构造检索式问答连

我们已经可以通过向量数据库找到最相关的内容了，接下来我们可以让 LLM 来用这些相关的内容回答我们的问题。

基于 LangChain，我们可以构造一个使用 GPT3.5 进行问答的检索式问答链，这是一种通过检索步骤进行问答的方法。我们可以通过传入一个语言模型和一个向量数据库来创建它作为检索器。然后，我们可以用问题作为查询调用它，得到一个答案。

In [ ]:
# 导入检索式问答链
from langchain.chains import RetrievalQA

In [ ]:
# 声明一个检索式问答链
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [ ]:
# 可以以该方式进行检索问答
question = "这节课的主要话题是什么"
result = qa_chain({"query": question})

In [ ]:
print(result["result"])

对于 LLM 来说，prompt 可以让更好的发挥大模型的能力。


我们首先定义了一个提示模板。它包含一些关于如何使用下面的上下文片段的说明，然后有一个上下文变量的占位符。

In [ ]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
{context}
问题：{question}
有用的回答："""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
# 中文版
question = "机器学习是其中一节的话题吗"

In [ ]:
result = qa_chain({"query": question})

In [ ]:
print(f"LLM 对问题的回答：{result["result"]}")

In [ ]:
print(f"向量数据库检索到的最相关的文档：{result["source_documents"][0]}")

这种方法非常好，因为它只涉及对语言模型的一次调用。然而，它也有局限性，即如果文档太多，可能无法将它们全部适配到上下文窗口中。

langchain 提供了几种不同的处理文档的方法：

|     类型      |                                定义/区别                                |                              优点                              |                              缺点                              |
|-------------|---------------------------------------------------------------------|----------------------------------------------------------------|----------------------------------------------------------------|
|   Stuff     | 将整个文本内容一次性输入给大模型进行处理。                               | - 只调用大模型一次，节省计算资源和时间。<br>- 上下文信息完整，有助于理解整体语义。<br>- 适用于处理较短的文本内容。 | - 不适用于处理较长的文本内容，可能导致模型过载。                |
|   Refine    | 通过多次调用大模型逐步改进文本质量，进行多次迭代优化。                          | - 可以在多次迭代中逐步改进文本质量。<br>- 适用于需要进行多次迭代优化的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要多轮迭代才能达到期望的文本质量。<br>- 不适用于实时性要求较高的场景。 |
| Map reduce  | 将大模型应用于每个文档，并将输出作为新文档传递给另一个模型，最终得到单个输出。               | - 可以对多个文档进行并行处理，提高处理效率。<br>- 可以通过多次迭代处理实现优化。<br>- 适用于需要对多个文档进行处理和合并的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要多轮迭代才能达到期望的结果。<br>- 不适用于处理单个文档的场景。 |
| Map re-rank | 在每个文档上运行初始提示，为答案给出一个分数，返回得分最高的响应。                        | - 可以根据置信度对文档进行排序和选择，提高结果的准确性。<br>- 可以提供更可靠的答案。<br>- 适用于需要根据置信度对文档进行排序和选择的场景。 | - 增加了计算资源和时间的消耗。<br>- 可能需要对多个文档进行评分和排序。<br>- 不适用于不需要对文档进行排序和选择的场景。 |

我们可以根据需要配置 chain_type 的参数，选择对应的处理方式。如：
```
RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)
```